# 🏗️ Data Mesh & Data Contracts

Bienvenue dans ce module où tu vas découvrir les architectures **Data Mesh** et les **Data Contracts** — deux concepts essentiels pour scaler les organisations data. Tu apprendras à passer d'une architecture centralisée à une approche décentralisée où chaque domaine est responsable de ses données.

---

## Prérequis

| Niveau | Compétence |
|--------|------------|
| ✅ Requis | Data Lakehouse (M20) |
| ✅ Requis | Data Quality (M23) |
| ✅ Requis | Orchestration (M22, M28) |
| 💡 Recommandé | Expérience en organisation data |

## 🎯 Objectifs du module

À la fin de ce module, tu seras capable de :

- Comprendre les 4 principes du Data Mesh
- Concevoir des **Data Products** avec ownership clair
- Écrire des **Data Contracts** complets
- Implémenter la validation de contracts avec **Soda**
- Configurer **DataHub** comme Data Catalog
- Gérer la **Schema Evolution** sans casser les consommateurs
- Mettre en place une **gouvernance fédérée**

---

## 1. Introduction au Data Mesh

### 1.1 Problèmes de l'Architecture Centralisée

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                    ARCHITECTURE CENTRALISÉE (MONOLITHE)                     │
│                                                                             │
│   ┌─────────┐                                                               │
│   │ Sales   │──┐                                                            │
│   │ Domain  │  │                                                            │
│   └─────────┘  │      ┌──────────────────┐      ┌─────────────────┐        │
│   ┌─────────┐  │      │                  │      │                 │        │
│   │Marketing│──┼─────▶│  Central Data    │─────▶│   Data Lake/    │        │
│   │ Domain  │  │      │     Team         │      │   Warehouse     │        │
│   └─────────┘  │      │                  │      │                 │        │
│   ┌─────────┐  │      │  (bottleneck!)   │      └─────────────────┘        │
│   │ Finance │──┘      └──────────────────┘                                  │
│   │ Domain  │                                                               │
│   └─────────┘                                                               │
│                                                                             │
│   PROBLÈMES :                                                               │
│   ❌ Équipe data = bottleneck (tout passe par eux)                          │
│   ❌ Pas de ownership clair ("c'est pas mes données")                       │
│   ❌ Time-to-market lent (file d'attente de demandes)                       │
│   ❌ Équipe data ne connaît pas le contexte métier                          │
│   ❌ Scalabilité organisationnelle limitée                                  │
└─────────────────────────────────────────────────────────────────────────────┘
```

### 1.2 Les 4 Principes du Data Mesh

Le **Data Mesh** est une approche architecturale proposée par **Zhamak Dehghani** (Thoughtworks) en 2019.

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                    LES 4 PRINCIPES DU DATA MESH                             │
│                                                                             │
│   ┌─────────────────────────────────────────────────────────────────────┐  │
│   │  1️⃣ DOMAIN OWNERSHIP                                                 │  │
│   │     Chaque domaine métier est responsable de ses données             │  │
│   │     → Sales gère les données sales, Marketing gère les données mktg │  │
│   └─────────────────────────────────────────────────────────────────────┘  │
│                                                                             │
│   ┌─────────────────────────────────────────────────────────────────────┐  │
│   │  2️⃣ DATA AS A PRODUCT                                                │  │
│   │     Les données sont traitées comme un produit avec des clients      │  │
│   │     → SLOs, documentation, support, qualité                          │  │
│   └─────────────────────────────────────────────────────────────────────┘  │
│                                                                             │
│   ┌─────────────────────────────────────────────────────────────────────┐  │
│   │  3️⃣ SELF-SERVE DATA PLATFORM                                         │  │
│   │     Une plateforme qui permet aux domaines d'être autonomes          │  │
│   │     → Infra, outils, templates, pas besoin de l'équipe centrale      │  │
│   └─────────────────────────────────────────────────────────────────────┘  │
│                                                                             │
│   ┌─────────────────────────────────────────────────────────────────────┐  │
│   │  4️⃣ FEDERATED COMPUTATIONAL GOVERNANCE                               │  │
│   │     Gouvernance décentralisée avec standards globaux                 │  │
│   │     → Policies automatisées, interopérabilité, compliance            │  │
│   └─────────────────────────────────────────────────────────────────────┘  │
└─────────────────────────────────────────────────────────────────────────────┘
```

### 1.3 Data Mesh vs Data Lake vs Data Lakehouse

| Aspect | Data Lake | Data Lakehouse | Data Mesh |
|--------|-----------|----------------|----------|
| **Type** | Architecture technique | Architecture technique | Architecture organisationnelle |
| **Focus** | Stockage | Stockage + Analytics | Organisation + Ownership |
| **Centralisation** | Centralisé | Centralisé | Décentralisé |
| **Ownership** | Équipe data | Équipe data | Domaines métier |
| **Compatibilité** | — | — | Peut utiliser Lakehouse underneath |

> ⚠️ **Important** : Data Mesh n'est PAS une technologie, c'est une approche organisationnelle. Tu peux implémenter un Data Mesh avec un Data Lakehouse comme infrastructure.

### 1.4 Quand Adopter le Data Mesh ?

**✅ Data Mesh est adapté si :**
- Organisation large (>100 personnes dans la data)
- Plusieurs domaines métier distincts
- Équipe data centrale = bottleneck avéré
- Domaines ont des équipes techniques capables
- Culture d'ownership et d'autonomie

**❌ Data Mesh n'est PAS adapté si :**
- Petite organisation (<20 personnes data)
- Un seul domaine métier
- Pas assez de maturité technique dans les domaines
- Besoin de contrôle centralisé fort
- Ressources limitées pour la plateforme

---

## 2. Domain Ownership & Data Products

### 2.1 Identifier les Domaines

Un **domaine** correspond généralement à un **bounded context** (DDD - Domain-Driven Design).

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                    EXEMPLE : E-COMMERCE                                     │
│                                                                             │
│   ┌─────────────┐   ┌─────────────┐   ┌─────────────┐   ┌─────────────┐   │
│   │   ORDERS    │   │  CUSTOMERS  │   │  PRODUCTS   │   │  MARKETING  │   │
│   │   Domain    │   │   Domain    │   │   Domain    │   │   Domain    │   │
│   │             │   │             │   │             │   │             │   │
│   │ - orders    │   │ - customers │   │ - products  │   │ - campaigns │   │
│   │ - payments  │   │ - addresses │   │ - inventory │   │ - emails    │   │
│   │ - refunds   │   │ - segments  │   │ - pricing   │   │ - analytics │   │
│   └─────────────┘   └─────────────┘   └─────────────┘   └─────────────┘   │
│                                                                             │
│   Chaque domaine :                                                          │
│   ✓ A son équipe dédiée                                                    │
│   ✓ Possède ses données                                                    │
│   ✓ Expose des Data Products                                               │
└─────────────────────────────────────────────────────────────────────────────┘
```

### 2.2 Qu'est-ce qu'un Data Product ?

Un **Data Product** est un ensemble de données exposé par un domaine pour être consommé par d'autres.

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                         DATA PRODUCT                                        │
│                                                                             │
│   ┌─────────────────────────────────────────────────────────────────────┐  │
│   │                      orders_fact                                    │  │
│   │                                                                     │  │
│   │   ┌─────────────┐   ┌─────────────┐   ┌─────────────┐              │  │
│   │   │   DATA      │   │   CODE      │   │  METADATA   │              │  │
│   │   │             │   │             │   │             │              │  │
│   │   │ Delta Table │   │ Spark Jobs  │   │ Schema      │              │  │
│   │   │ (storage)   │   │ Airflow DAG │   │ Contract    │              │  │
│   │   │             │   │ dbt models  │   │ SLOs        │              │  │
│   │   └─────────────┘   └─────────────┘   │ Docs        │              │  │
│   │                                       └─────────────┘              │  │
│   │                                                                     │  │
│   │   ┌─────────────────────────────────────────────────────────────┐  │  │
│   │   │                    OUTPUT PORTS                             │  │  │
│   │   │   SQL (Snowflake) │ API (REST) │ File (S3) │ Stream (Kafka) │  │  │
│   │   └─────────────────────────────────────────────────────────────┘  │  │
│   └─────────────────────────────────────────────────────────────────────┘  │
│                                                                             │
│   Owner: orders-team@company.com                                           │
│   SLA: 99.9% availability, <1h freshness                                   │
└─────────────────────────────────────────────────────────────────────────────┘
```

### 2.3 Caractéristiques d'un Bon Data Product (DATSIS)

| Caractéristique | Description | Exemple |
|-----------------|-------------|--------|
| **D**iscoverable | Facile à trouver | Référencé dans DataHub |
| **A**ddressable | Accès standardisé | `snowflake://prod.orders.orders_fact` |
| **T**rustworthy | Qualité garantie | SLOs, tests automatisés |
| **S**elf-describing | Documentation complète | Schema, descriptions, exemples |
| **I**nteroperable | Standards communs | Formats, naming conventions |
| **S**ecure | Accès contrôlé | RBAC, encryption, audit |

### 2.4 Équipe Data Product

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                    ÉQUIPE DATA PRODUCT (par domaine)                        │
│                                                                             │
│   ┌─────────────────────────────────────────────────────────────────────┐  │
│   │                                                                     │  │
│   │   Product Owner ─── Définit les priorités, roadmap                  │  │
│   │        │                                                            │  │
│   │        ├── Data Engineer ─── Pipelines, infrastructure              │  │
│   │        │                                                            │  │
│   │        ├── Analytics Engineer ─── Modèles, transformations          │  │
│   │        │                                                            │  │
│   │        └── Domain Expert ─── Connaissances métier, validation       │  │
│   │                                                                     │  │
│   └─────────────────────────────────────────────────────────────────────┘  │
│                                                                             │
│   Responsabilités :                                                         │
│   ✓ Développer et maintenir les Data Products du domaine                   │
│   ✓ Garantir la qualité (SLOs)                                             │
│   ✓ Supporter les consommateurs                                            │
│   ✓ Documenter et communiquer les changements                              │
└─────────────────────────────────────────────────────────────────────────────┘
```

---

## 3. Self-Serve Data Platform

### 3.1 Rôle de la Plateforme

La plateforme est un **enabler**, pas un bottleneck. Elle fournit les outils pour que les domaines soient autonomes.

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                    SELF-SERVE DATA PLATFORM                                 │
│                                                                             │
│   ┌─────────────────────────────────────────────────────────────────────┐  │
│   │                    DOMAIN TEAMS                                     │  │
│   │   Orders │ Customers │ Products │ Marketing │ Finance              │  │
│   └────────────────────────────┬────────────────────────────────────────┘  │
│                                │                                            │
│                                ▼                                            │
│   ┌─────────────────────────────────────────────────────────────────────┐  │
│   │                    PLATFORM SERVICES                                │  │
│   │                                                                     │  │
│   │   ┌─────────────┐  ┌─────────────┐  ┌─────────────┐                │  │
│   │   │ Data Product│  │   Data      │  │   Access    │                │  │
│   │   │  Templates  │  │  Catalog    │  │ Management  │                │  │
│   │   │             │  │  (DataHub)  │  │   (RBAC)    │                │  │
│   │   └─────────────┘  └─────────────┘  └─────────────┘                │  │
│   │                                                                     │  │
│   │   ┌─────────────┐  ┌─────────────┐  ┌─────────────┐                │  │
│   │   │   Quality   │  │ Observability│  │  Contract  │                │  │
│   │   │   (Soda)    │  │ (Monitoring) │  │ Validation │                │  │
│   │   └─────────────┘  └─────────────┘  └─────────────┘                │  │
│   └─────────────────────────────────────────────────────────────────────┘  │
│                                │                                            │
│                                ▼                                            │
│   ┌─────────────────────────────────────────────────────────────────────┐  │
│   │                    INFRASTRUCTURE                                   │  │
│   │   Compute (Spark) │ Storage (S3/Delta) │ Orchestration (Airflow)   │  │
│   └─────────────────────────────────────────────────────────────────────┘  │
└─────────────────────────────────────────────────────────────────────────────┘
```

### 3.2 Composants Essentiels

| Composant | Fonction | Outils |
|-----------|----------|--------|
| **Infrastructure** | Compute, storage, networking | Kubernetes, S3, Snowflake |
| **Templates** | Bootstrapper un Data Product | Cookiecutter, Backstage |
| **Catalog** | Découverte, lineage, metadata | DataHub, OpenMetadata, Atlan |
| **Access Management** | RBAC, policies | Unity Catalog, Ranger |
| **Quality** | Tests, monitoring | Soda, Great Expectations |
| **Observability** | Métriques, alertes | Datadog, Monte Carlo |

### 3.3 Technologies de Data Catalog

| Outil | Type | Points forts |
|-------|------|-------------|
| **DataHub** | Open Source | Lineage, search, extensible |
| **OpenMetadata** | Open Source | UI moderne, collaboration |
| **Atlan** | Commercial | UX, collaboration, governance |
| **Unity Catalog** | Databricks | Intégré Databricks, ACL |
| **AWS Glue Catalog** | AWS | Intégré AWS, serverless |

---

## 4. Data Contracts

### 4.1 Pourquoi les Data Contracts ?

Sans contracts, c'est le chaos :

```text
❌ SANS DATA CONTRACT :

Producer: "J'ai renommé la colonne 'user_id' en 'customer_id', c'était plus clair."
Consumer: "Tu as cassé tous nos dashboards !" 😡
Producer: "C'est pas ma faute si vous dépendez de mes données sans me prévenir."

✅ AVEC DATA CONTRACT :

Producer: "Je veux renommer 'user_id' en 'customer_id'."
Contract: "Breaking change détecté. 3 consommateurs impactés."
Producer: "OK, je crée une nouvelle version avec période de deprecation."
```

### 4.2 Anatomie d'un Data Contract

Un Data Contract définit l'**interface** entre un producteur et ses consommateurs.

In [ ]:
# Exemple de Data Contract complet (YAML)

data_contract_yaml = """
# ═══════════════════════════════════════════════════════════════
# DATA CONTRACT : orders_fact
# ═══════════════════════════════════════════════════════════════

dataContractSpecification: 0.9.3
id: orders-domain.orders-fact

# ─────────────────────────────────────────────────────────────────
# INFO : Métadonnées générales
# ─────────────────────────────────────────────────────────────────
info:
  title: Orders Fact Table
  version: 1.0.0
  status: active  # draft | active | deprecated | retired
  description: |
    Table de faits contenant toutes les commandes.
    Une ligne par commande. Mise à jour en near real-time.
  owner: orders-team@company.com
  contact:
    slack: "#orders-data"
    oncall: "https://pagerduty.com/orders-data"

# ─────────────────────────────────────────────────────────────────
# SCHEMA : Structure des données
# ─────────────────────────────────────────────────────────────────
schema:
  type: table
  fields:
    - name: order_id
      type: string
      required: true
      unique: true
      description: Identifiant unique de la commande (UUID v4)
      example: "550e8400-e29b-41d4-a716-446655440000"
      pii: false
      
    - name: customer_id
      type: string
      required: true
      description: Référence au domaine customers
      pii: true
      
    - name: order_date
      type: timestamp
      required: true
      description: Date et heure de la commande (UTC)
      
    - name: amount
      type: decimal(10,2)
      required: true
      description: Montant total en EUR
      constraints:
        minimum: 0
        maximum: 1000000
        
    - name: status
      type: string
      required: true
      description: Statut actuel de la commande
      enum:
        - pending
        - confirmed
        - shipped
        - delivered
        - cancelled
        
    - name: shipping_country
      type: string
      required: false
      description: Code pays ISO 3166-1 alpha-2
      pattern: "^[A-Z]{2}$"

# ─────────────────────────────────────────────────────────────────
# SEMANTICS : Signification métier
# ─────────────────────────────────────────────────────────────────
semantics:
  granularity: one row per order
  temporality: event time (order_date)
  updateFrequency: near real-time (< 15 min)
  businessRules:
    - "amount = somme des lignes + taxes - remises"
    - "Les commandes annulées restent dans la table (status=cancelled)"
    - "order_date est en UTC, pas en heure locale"

# ─────────────────────────────────────────────────────────────────
# QUALITY : SLOs de qualité
# ─────────────────────────────────────────────────────────────────
quality:
  freshness:
    threshold: 1 hour
    column: order_date
    
  completeness:
    - column: order_id
      threshold: 100%
    - column: customer_id
      threshold: 100%
    - column: amount
      threshold: 99.9%
      
  validity:
    - column: amount
      rule: ">= 0"
      threshold: 100%
    - column: status
      rule: "in enum values"
      threshold: 100%
      
  uniqueness:
    - column: order_id
      threshold: 100%

# ─────────────────────────────────────────────────────────────────
# SLA : Service Level Agreements
# ─────────────────────────────────────────────────────────────────
sla:
  availability: 99.9%
  latency: "< 5 minutes from source"
  retention: 7 years
  supportHours: "24/7"
  incidentResponse: "< 1 hour"

# ─────────────────────────────────────────────────────────────────
# ACCESS : Qui peut accéder
# ─────────────────────────────────────────────────────────────────
access:
  classification: internal
  location: "snowflake://prod.orders.orders_fact"
  consumers:
    - team: analytics
      purpose: dashboards
    - team: marketing
      purpose: segmentation
    - team: finance
      purpose: revenue reporting
"""

print("📄 Data Contract Example:")
print(data_contract_yaml)

### 4.3 DataContract CLI (Open Source)

```bash
# Installation
pip install datacontract-cli

# Initialiser un nouveau contract
datacontract init orders-fact

# Valider la syntaxe
datacontract lint datacontract.yaml

# Tester contre des données réelles
datacontract test datacontract.yaml \
  --source snowflake://prod.orders.orders_fact

# Générer la documentation HTML
datacontract export datacontract.yaml --format html > docs/orders.html

# Détecter les breaking changes
datacontract diff v1/datacontract.yaml v2/datacontract.yaml
```

### 4.4 Validation avec Soda

In [ ]:
# Soda checks générés depuis le Data Contract

soda_checks_yaml = """
# soda_checks.yaml
# Généré depuis le Data Contract orders-domain.orders-fact

checks for orders_fact:
  # ─────────────────────────────────────────────────────────────
  # Freshness : données < 1 heure
  # ─────────────────────────────────────────────────────────────
  - freshness(order_date) < 1h:
      name: "Data freshness SLO"
      fail: when > 1h
      warn: when > 30m

  # ─────────────────────────────────────────────────────────────
  # Completeness : pas de nulls sur colonnes required
  # ─────────────────────────────────────────────────────────────
  - missing_count(order_id) = 0:
      name: "order_id completeness"
      
  - missing_count(customer_id) = 0:
      name: "customer_id completeness"
      
  - missing_percent(amount) < 0.1:
      name: "amount completeness (99.9%)"

  # ─────────────────────────────────────────────────────────────
  # Validity : valeurs dans les plages attendues
  # ─────────────────────────────────────────────────────────────
  - invalid_count(amount) = 0:
      name: "amount must be >= 0"
      valid min: 0
      
  - invalid_count(status) = 0:
      name: "status must be in enum"
      valid values: [pending, confirmed, shipped, delivered, cancelled]
      
  - invalid_count(shipping_country) = 0:
      name: "shipping_country format"
      valid regex: "^[A-Z]{2}$"

  # ─────────────────────────────────────────────────────────────
  # Uniqueness : pas de doublons
  # ─────────────────────────────────────────────────────────────
  - duplicate_count(order_id) = 0:
      name: "order_id uniqueness"

  # ─────────────────────────────────────────────────────────────
  # Volume : anomaly detection
  # ─────────────────────────────────────────────────────────────
  - anomaly detection for row_count:
      name: "Volume anomaly detection"
      warn: when diff > 20%
      fail: when diff > 50%
"""

print("✅ Soda Checks (from Data Contract):")
print(soda_checks_yaml)

### 4.5 Exécuter Soda

```bash
# Installation
pip install soda-core-spark  # ou soda-core-snowflake, soda-core-postgres

# Configuration (soda_config.yaml)
# data_source my_warehouse:
#   type: snowflake
#   account: mycompany.us-east-1
#   username: ${SNOWFLAKE_USER}
#   password: ${SNOWFLAKE_PASSWORD}
#   database: PROD
#   schema: ORDERS

# Exécuter les checks
soda scan -d my_warehouse -c soda_config.yaml soda_checks.yaml

# Résultat :
# Scan summary:
# 8/8 checks PASSED
# 0 checks WARNED
# 0 checks FAILED
```

### 4.6 Contract Validation dans CI/CD

```yaml
# .github/workflows/contract-validation.yaml
name: Data Contract Validation

on:
  push:
    paths:
      - 'contracts/**'
  schedule:
    - cron: '0 * * * *'  # Toutes les heures

jobs:
  validate:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      
      - name: Install tools
        run: pip install datacontract-cli soda-core
        
      - name: Lint contracts
        run: datacontract lint contracts/*.yaml
        
      - name: Check for breaking changes
        run: |
          datacontract diff contracts/orders.yaml \
            --against main:contracts/orders.yaml
            
      - name: Run Soda checks
        run: soda scan -d warehouse soda_checks.yaml
        
      - name: Alert on failure
        if: failure()
        run: |
          curl -X POST $SLACK_WEBHOOK \
            -d '{"text": "🚨 Data Contract validation failed!"}'
```

---

## 5. Schema Evolution & Versioning

### 5.1 Breaking vs Non-Breaking Changes

| Type de changement | Breaking ? | Action recommandée |
|-------------------|------------|--------------------|
| Ajouter colonne **nullable** | ❌ Non | ✅ OK direct |
| Ajouter colonne **required** | ✅ Oui | Nouvelle version majeure |
| Renommer colonne | ✅ Oui | Deprecation + alias |
| Changer type (ex: string→int) | ✅ Oui | Nouvelle version majeure |
| Supprimer colonne | ✅ Oui | Deprecation period (30-90 jours) |
| Changer enum (ajouter valeur) | ⚠️ Dépend | Prévenir les consommateurs |
| Changer contrainte (min/max) | ⚠️ Dépend | Prévenir les consommateurs |

### 5.2 Stratégies de Versioning

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                    SEMANTIC VERSIONING                                      │
│                                                                             │
│                         MAJOR.MINOR.PATCH                                   │
│                           │     │     │                                     │
│                           │     │     └── Bug fixes, no schema change       │
│                           │     └──────── New features, backward compatible │
│                           └────────────── Breaking changes                  │
│                                                                             │
│   Exemples :                                                                │
│   1.0.0 → 1.0.1 : Fix bug dans calcul                                      │
│   1.0.1 → 1.1.0 : Ajout colonne nullable                                   │
│   1.1.0 → 2.0.0 : Renommage colonne (breaking)                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

### 5.3 Backward vs Forward Compatibility

| Type | Description | Qui peut lire quoi |
|------|-------------|-------------------|
| **Backward** | Nouveau code lit ancien format | ✅ v2 reader → v1 data |
| **Forward** | Ancien code lit nouveau format | ✅ v1 reader → v2 data |
| **Full** | Les deux | ✅ Best practice |

### 5.4 Processus de Deprecation

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                    PROCESSUS DE DEPRECATION                                 │
│                                                                             │
│   Jour 0          Jour 30         Jour 60         Jour 90                  │
│     │               │               │               │                       │
│     ▼               ▼               ▼               ▼                       │
│   ┌─────┐         ┌─────┐         ┌─────┐         ┌─────┐                  │
│   │Annonce│───────│Rappel│───────│Warning│───────│Remove│                  │
│   │       │       │      │        │       │       │      │                  │
│   └─────┘         └─────┘         └─────┘         └─────┘                  │
│                                                                             │
│   • Annoncer la deprecation dans le contract                               │
│   • Notifier tous les consommateurs                                        │
│   • Fournir un guide de migration                                          │
│   • Logger les accès à la colonne deprecated                               │
│   • Supprimer après la période                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

### 5.5 Exemple de Deprecation dans le Contract

```yaml
schema:
  fields:
    - name: user_id
      type: string
      deprecated: true
      deprecationDate: "2024-01-15"
      removalDate: "2024-04-15"
      replacedBy: customer_id
      description: "[DEPRECATED] Use customer_id instead"
      
    - name: customer_id
      type: string
      required: true
      description: "Unique customer identifier (replaces user_id)"
```

---

## 6. Data Catalog & Discovery

### 6.1 Pourquoi un Data Catalog ?

| Problème | Solution Catalog |
|----------|------------------|
| "Où trouver les données clients ?" | **Search** |
| "D'où viennent ces données ?" | **Lineage** |
| "Qui est responsable de cette table ?" | **Ownership** |
| "Cette colonne contient quoi exactement ?" | **Documentation** |
| "Les données sont-elles fiables ?" | **Quality scores** |

### 6.2 Types de Metadata

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                    TYPES DE METADATA                                        │
│                                                                             │
│   ┌─────────────────────────────────────────────────────────────────────┐  │
│   │  TECHNICAL METADATA                                                 │  │
│   │  • Schema (colonnes, types)                                         │  │
│   │  • Lineage (sources, transformations)                               │  │
│   │  • Partitioning, format, location                                   │  │
│   └─────────────────────────────────────────────────────────────────────┘  │
│                                                                             │
│   ┌─────────────────────────────────────────────────────────────────────┐  │
│   │  BUSINESS METADATA                                                  │  │
│   │  • Descriptions, définitions                                        │  │
│   │  • Ownership, contact                                               │  │
│   │  • Business glossary terms                                          │  │
│   │  • Classification (PII, confidential)                               │  │
│   └─────────────────────────────────────────────────────────────────────┘  │
│                                                                             │
│   ┌─────────────────────────────────────────────────────────────────────┐  │
│   │  OPERATIONAL METADATA                                               │  │
│   │  • Freshness, last update                                           │  │
│   │  • Quality scores                                                   │  │
│   │  • Usage statistics                                                 │  │
│   │  • Query patterns                                                   │  │
│   └─────────────────────────────────────────────────────────────────────┘  │
└─────────────────────────────────────────────────────────────────────────────┘
```

### 6.3 DataHub : Setup et Utilisation

**DataHub** est le Data Catalog open source le plus populaire (créé par LinkedIn).

```bash
# Installation avec Docker
git clone https://github.com/datahub-project/datahub.git
cd datahub/docker
./quickstart.sh

# Accéder à l'UI
# http://localhost:9002
# Login: datahub / datahub
```

### 6.4 Ingestion de Metadata

```yaml
# recipes/snowflake_ingestion.yaml
source:
  type: snowflake
  config:
    account_id: "mycompany.us-east-1"
    username: ${SNOWFLAKE_USER}
    password: ${SNOWFLAKE_PASSWORD}
    warehouse: COMPUTE_WH
    database_pattern:
      allow:
        - "PROD"
    include_table_lineage: true
    profiling:
      enabled: true

sink:
  type: datahub-rest
  config:
    server: "http://localhost:8080"
```

```bash
# Installer le CLI et exécuter l'ingestion
pip install 'acryl-datahub[snowflake]'
datahub ingest -c recipes/snowflake_ingestion.yaml
```

### 6.5 Lineage Automatique

```python
# Spark avec DataHub Lineage
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("OrdersETL") \
    .config("spark.extraListeners", "datahub.spark.DatahubSparkListener") \
    .config("spark.datahub.rest.server", "http://localhost:8080") \
    .getOrCreate()

# Ces opérations créent automatiquement le lineage dans DataHub
orders = spark.read.parquet("s3://bucket/raw/orders")
customers = spark.read.parquet("s3://bucket/dim/customers")

enriched = orders.join(customers, "customer_id")
enriched.write.parquet("s3://bucket/gold/orders_enriched")

# Lineage capturé :
# raw/orders ──┐
#              ├──▶ gold/orders_enriched
# dim/customers┘
```

### 6.6 Alternatives à DataHub

| Outil | Type | Points forts |
|-------|------|-------------|
| **DataHub** | Open Source | Lineage, extensible, actif |
| **OpenMetadata** | Open Source | UI moderne, tout-en-un |
| **Atlan** | Commercial | UX, collaboration |
| **Alation** | Commercial | Recherche, ML-powered |
| **Unity Catalog** | Databricks | Intégré, ACL avancé |

---

## 7. Federated Governance

### 7.1 Gouvernance Centralisée vs Fédérée

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                    GOUVERNANCE CENTRALISÉE vs FÉDÉRÉE                       │
│                                                                             │
│   CENTRALISÉE                         FÉDÉRÉE (Data Mesh)                   │
│   ───────────                         ───────────────────                   │
│                                                                             │
│   ┌─────────────────┐                 ┌─────────────────┐                  │
│   │  Central Team   │                 │  Global Policies│                  │
│   │  decides ALL    │                 │  (standards)    │                  │
│   │  policies       │                 └────────┬────────┘                  │
│   └────────┬────────┘                          │                           │
│            │                                   ▼                           │
│            ▼                          ┌───────────────────┐                │
│   ┌─────────────────┐                 │   Domain A        │                │
│   │  All Domains    │                 │   (local policies)│                │
│   │  follow blindly │                 ├───────────────────┤                │
│   └─────────────────┘                 │   Domain B        │                │
│                                       │   (local policies)│                │
│   ❌ Slow                             └───────────────────┘                │
│   ❌ One-size-fits-all                                                     │
│   ❌ Bottleneck                       ✅ Fast                               │
│                                       ✅ Context-aware                      │
│                                       ✅ Scalable                           │
└─────────────────────────────────────────────────────────────────────────────┘
```

### 7.2 Policies Globales vs Locales

| Scope | Exemples | Qui décide |
|-------|----------|------------|
| **Global** | Naming conventions, formats de dates, compliance GDPR | Platform team |
| **Local** | Schema spécifique, SLOs, refresh frequency | Domain team |

### 7.3 Standards Communs (Interopérabilité)

```yaml
# global_standards.yaml
# Standards que TOUS les domaines doivent respecter

naming:
  tables: "{domain}_{entity}_{type}"  # orders_customers_dim
  columns: snake_case
  timestamps: "*_at" suffix  # created_at, updated_at

formats:
  dates: ISO 8601 (YYYY-MM-DD)
  timestamps: ISO 8601 with timezone (UTC)
  currency: ISO 4217 codes (EUR, USD)
  country: ISO 3166-1 alpha-2 (FR, US)

quality:
  minimumFreshness: 24h
  minimumCompleteness: 95%
  requiredTests:
    - uniqueness on primary key
    - not null on required columns

compliance:
  piiColumns:
    mustBeTagged: true
    defaultRetention: 3 years
    encryption: required
```

### 7.4 Data Quality as Code

```python
# Policies automatisées via CI/CD

def validate_global_standards(contract):
    """Valider qu'un contract respecte les standards globaux."""
    errors = []
    
    # 1. Naming convention
    if not re.match(r'^[a-z]+_[a-z]+_(fact|dim|event)$', contract['info']['title']):
        errors.append("Table name must follow {domain}_{entity}_{type} convention")
    
    # 2. Required metadata
    if 'owner' not in contract['info']:
        errors.append("Owner is required")
    
    # 3. Quality SLOs
    if contract.get('quality', {}).get('freshness', {}).get('threshold', '999h') > '24h':
        errors.append("Freshness SLO must be <= 24h")
    
    # 4. PII tagging
    for field in contract.get('schema', {}).get('fields', []):
        if 'pii' not in field:
            errors.append(f"Field {field['name']} must have PII tag")
    
    return errors
```

### 7.5 Compliance (GDPR, PII)

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                    PII MANAGEMENT                                           │
│                                                                             │
│   1. IDENTIFICATION                                                         │
│      Tag PII columns in contract: pii: true                                │
│                                                                             │
│   2. CLASSIFICATION                                                         │
│      • Direct PII: email, phone, SSN                                       │
│      • Indirect PII: customer_id, IP address                               │
│      • Sensitive: health, financial, political                             │
│                                                                             │
│   3. PROTECTION                                                             │
│      • Encryption at rest                                                  │
│      • Access control (need-to-know)                                       │
│      • Masking for non-prod environments                                   │
│                                                                             │
│   4. RETENTION                                                              │
│      • Define retention period per classification                          │
│      • Automated deletion                                                  │
│      • Right to be forgotten (GDPR)                                        │
└─────────────────────────────────────────────────────────────────────────────┘
```

---

## 8. Implémentation Pratique

### 8.1 Par Où Commencer ?

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                    ROADMAP D'ADOPTION DATA MESH                             │
│                                                                             │
│   PHASE 1: PILOT (3-6 mois)                                                │
│   ─────────────────────────                                                │
│   • Choisir 1-2 domaines pilotes                                           │
│   • Définir les premiers Data Products                                     │
│   • Écrire les premiers Data Contracts                                     │
│   • Setup basique du catalog (DataHub)                                     │
│                                                                             │
│   PHASE 2: EXPAND (6-12 mois)                                              │
│   ────────────────────────────                                             │
│   • Ajouter 3-5 domaines                                                   │
│   • Développer la self-serve platform                                      │
│   • Automatiser la validation des contracts                                │
│   • Définir les standards globaux                                          │
│                                                                             │
│   PHASE 3: SCALE (12+ mois)                                                │
│   ─────────────────────────                                                │
│   • Tous les domaines on-boardés                                           │
│   • Plateforme mature et self-serve                                        │
│   • Gouvernance fédérée opérationnelle                                     │
│   • Métriques et amélioration continue                                     │
└─────────────────────────────────────────────────────────────────────────────┘
```

### 8.2 Critères de Sélection des Domaines Pilotes

| Critère | Bon candidat | Mauvais candidat |
|---------|--------------|------------------|
| **Maturité équipe** | Équipe tech capable | Pas de compétences data |
| **Complexité** | Modérée | Trop simple ou trop complexe |
| **Visibilité** | Impact business visible | Projet interne invisible |
| **Dépendances** | Peu de dépendances | Dépend de tout le monde |
| **Sponsor** | Management engagé | Pas de sponsor |

### 8.3 Anti-Patterns à Éviter

| Anti-Pattern | Problème | Solution |
|--------------|----------|----------|
| **Big Bang** | Tout migrer d'un coup | Approche incrémentale |
| **Platform-first** | Construire la plateforme avant les besoins | Use-case driven |
| **No Governance** | Chaque domaine fait ce qu'il veut | Standards globaux |
| **Over-Governance** | Trop de règles, retour au bottleneck | Balance global/local |
| **Copy-Paste** | Copier les données au lieu de les consommer | Data Products vrais |

### 8.4 Métriques de Succès

| Métrique | Description | Target |
|----------|-------------|--------|
| **Time to Data** | Temps pour accéder à une nouvelle donnée | < 1 jour |
| **Data Product Count** | Nombre de Data Products publiés | Croissance |
| **Contract Coverage** | % de tables avec contract | > 80% |
| **Quality Score** | Score moyen de qualité | > 95% |
| **Consumer Satisfaction** | NPS des consommateurs | > 50 |

---

## 9. Exercices Pratiques

### Exercice 1 : Identifier les Domaines

Pour une entreprise de e-commerce avec les équipes suivantes :
- Équipe Produit (catalogue, pricing)
- Équipe Ventes (commandes, paiements)
- Équipe Logistique (livraisons, stock)
- Équipe Marketing (campagnes, analytics)
- Équipe Support (tickets, satisfaction)

**Questions :**
1. Identifier les 5 domaines Data Mesh
2. Lister 2-3 Data Products par domaine
3. Identifier les dépendances entre domaines

---

### Exercice 2 : Écrire un Data Contract

Écrire un Data Contract complet pour la table `customers_dim` avec :
- 6+ colonnes (id, name, email, country, created_at, segment)
- PII identifié
- SLOs de qualité
- SLA de disponibilité

---

### Exercice 3 : Setup DataHub Local

1. Installer DataHub avec Docker
2. Créer une recette d'ingestion pour un fichier Parquet local
3. Exécuter l'ingestion
4. Explorer le lineage dans l'UI

---

### Exercice 4 : Validation de Contract avec Soda

1. Convertir le Data Contract de l'exercice 2 en checks Soda
2. Créer des données de test (certaines valides, certaines invalides)
3. Exécuter Soda et interpréter les résultats

---

### Exercice 5 : Schema Evolution

Simuler une schema evolution :
1. Version 1.0.0 du contract `orders_fact`
2. Ajouter une colonne nullable `discount_code` (non-breaking)
3. Renommer `user_id` en `customer_id` (breaking)
4. Documenter le processus de deprecation

---

## 10. Mini-Projet : Data Mesh Pilot

### Objectif

Implémenter un pilot Data Mesh avec 2 domaines.

```text
┌─────────────────────────────────────────────────────────────────────────────┐
│                      MINI-PROJET : DATA MESH PILOT                          │
│                                                                             │
│   ┌─────────────────┐         ┌─────────────────┐                          │
│   │  ORDERS DOMAIN  │         │ CUSTOMERS DOMAIN│                          │
│   │                 │         │                 │                          │
│   │  Data Product:  │         │  Data Product:  │                          │
│   │  - orders_fact  │◀───────▶│  - customers_dim│                          │
│   │  - Contract ✓   │         │  - Contract ✓   │                          │
│   │  - SLOs ✓       │         │  - SLOs ✓       │                          │
│   └────────┬────────┘         └────────┬────────┘                          │
│            │                           │                                    │
│            ▼                           ▼                                    │
│   ┌─────────────────────────────────────────────────────────────────────┐  │
│   │                     SELF-SERVE PLATFORM                             │  │
│   │                                                                     │  │
│   │   ┌─────────────┐  ┌─────────────┐  ┌─────────────┐                │  │
│   │   │   DataHub   │  │   Contract  │  │   Quality   │                │  │
│   │   │   Catalog   │  │  Validation │  │  (Soda)     │                │  │
│   │   └─────────────┘  └─────────────┘  └─────────────┘                │  │
│   └─────────────────────────────────────────────────────────────────────┘  │
└─────────────────────────────────────────────────────────────────────────────┘
```

### Livrables

1. **Data Contracts** : 2 contracts complets (orders_fact, customers_dim)
2. **Soda Checks** : Validation automatisée pour chaque contract
3. **DataHub** : Catalog avec les 2 Data Products
4. **Lineage** : Visualisation des dépendances
5. **CI/CD** : Pipeline de validation des contracts
6. **Documentation** : README avec architecture et standards

### Structure du Projet

```text
data-mesh-pilot/
├── domains/
│   ├── orders/
│   │   ├── contracts/
│   │   │   └── orders_fact.yaml
│   │   ├── soda/
│   │   │   └── orders_checks.yaml
│   │   └── pipelines/
│   │       └── orders_etl.py
│   └── customers/
│       ├── contracts/
│       │   └── customers_dim.yaml
│       ├── soda/
│       │   └── customers_checks.yaml
│       └── pipelines/
│           └── customers_etl.py
├── platform/
│   ├── datahub/
│   │   └── docker-compose.yaml
│   ├── standards/
│   │   └── global_standards.yaml
│   └── ci/
│       └── contract_validation.yaml
├── data/
│   ├── orders.parquet
│   └── customers.parquet
└── README.md
```

### Critères de Succès

- [ ] 2 Data Contracts complets et valides
- [ ] Soda checks passent à 100%
- [ ] DataHub affiche les 2 Data Products
- [ ] Lineage visible entre orders et customers
- [ ] CI/CD détecte les breaking changes
- [ ] Documentation claire

---

## 📚 Ressources

### Documentation
- [Data Mesh Principles (Zhamak Dehghani)](https://martinfowler.com/articles/data-mesh-principles.html)
- [Data Contract Specification](https://datacontract.com/)
- [DataHub Documentation](https://datahubproject.io/docs/)
- [Soda Documentation](https://docs.soda.io/)

### Livres
- *Data Mesh: Delivering Data-Driven Value at Scale* — Zhamak Dehghani
- *Fundamentals of Data Engineering* — Joe Reis, Matt Housley

### Outils
- [DataHub](https://datahubproject.io/) — Data Catalog open source
- [OpenMetadata](https://open-metadata.org/) — Alternative open source
- [Soda](https://www.soda.io/) — Data Quality
- [DataContract CLI](https://cli.datacontract.com/) — Contract validation

---

## ➡️ Prochaine étape

👉 **Module suivant : `33_real_time_analytics`** — Real-Time Analytics

---

🎉 **Félicitations !** Tu maîtrises maintenant les concepts de Data Mesh et Data Contracts.